In [7]:
import pandas as pd
import numpy as np
import re

In [8]:
train = pd.read_csv('train_2kmZucJ.csv')
test = pd.read_csv('test_oJQbWVk.csv')
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [9]:
train.shape

(7920, 3)

In [10]:
train.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [11]:
train['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [14]:
neg = train[train['label']==1]
neg.head()

,id,label,tweet
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
10,11,1,hey #apple when you make a new ipod dont make ...
11,12,1,Ha! Not heavy machinery but it does what I nee...
12,13,1,Contemplating giving in to the iPhone bandwago...


In [15]:
pos = train[train['label']==0]
pos.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
6,7,0,Happy for us .. #instapic #instadaily #us #son...


In [16]:
train['tweet'] = train['tweet'].apply(lambda x: x.lower())
test['tweet'] = test['tweet'].apply(lambda x: x.lower())

In [17]:
import string
def clean_text(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [18]:
train['tweet'] = train['tweet'].apply(lambda x: clean_text(x))
test['tweet'] = test['tweet'].apply(lambda x: clean_text(x))

In [19]:
train.head()

,id,label,tweet
0,1,0,fingerprint pregnancy test android apps beaut...
1,2,0,finally a transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememories un...
3,4,0,im wired i know im george i was made that way ...
4,5,1,what amazing service apple wont even talk to m...


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Udhhav
[nltk_data]     Arora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [22]:
stopwords = stopwords.words('english')

In [24]:
def remove_stopwords(text):
    text = [item for item in text.split() if item not in stopwords]
    return ' '.join(text)

train['updated_tweet'] = train['tweet'].apply(remove_stopwords)
test['updated_tweet'] = test['tweet'].apply(remove_stopwords)

train.head()

,id,label,tweet,updated_tweet
0,1,0,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...
1,2,0,finally a transparant silicon case thanks to ...,finally transparant silicon case thanks uncle ...
2,3,0,we love this would you go talk makememories un...,love would go talk makememories unplug relax i...
3,4,0,im wired i know im george i was made that way ...,im wired know im george made way iphone cute d...
4,5,1,what amazing service apple wont even talk to m...,amazing service apple wont even talk question ...


In [25]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(text):
    text = [stemmer.stem(word) for word in text.split()]
    return ' '.join(text)

train['stemmed_tweet'] = train['updated_tweet'].apply(stemming)
test['stemmed_tweet'] = test['updated_tweet'].apply(stemming)
train.head()

,id,label,tweet,updated_tweet,stemmed_tweet
0,1,0,fingerprint pregnancy test android apps beaut...,fingerprint pregnancy test android apps beauti...,fingerprint pregnanc test android app beauti c...
1,2,0,finally a transparant silicon case thanks to ...,finally transparant silicon case thanks uncle ...,final transpar silicon case thank uncl yay son...
2,3,0,we love this would you go talk makememories un...,love would go talk makememories unplug relax i...,love would go talk makememori unplug relax iph...
3,4,0,im wired i know im george i was made that way ...,im wired know im george made way iphone cute d...,im wire know im georg made way iphon cute dave...
4,5,1,what amazing service apple wont even talk to m...,amazing service apple wont even talk question ...,amaz servic appl wont even talk question unles...


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='word',binary=True)
cv.fit(train['stemmed_tweet'])

trained = cv.fit_transform(train['stemmed_tweet'])
tested = cv.transform(test['stemmed_tweet'])

In [27]:
y=train['label']

In [28]:
from sklearn.svm import SVC
from sklearn import model_selection

model1 = SVC()
scores = model_selection.cross_val_score(model1,trained,y,cv=3,scoring='f1')

In [29]:
scores.mean()

0.756663712091739

In [30]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB(alpha=1)
scores = model_selection.cross_val_score(model2, trained, y, cv=3, scoring="f1")
scores.mean()

0.7865807257729228

In [31]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
scores = model_selection.cross_val_score(model3,trained,y,cv=3,scoring='f1')
scores.mean()

0.7578773543335432

In [32]:
model2.fit(trained,y)

MultinomialNB(alpha=1)

In [33]:
submission = pd.read_csv('sample_submission_LnhVWA4.csv')
submission.head()

,id,label
0,7921,0
1,7922,0
2,7923,0
3,7924,0
4,7925,0


In [34]:
submission['label'] = model2.predict(tested)
submission.head()

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1


In [35]:
submission.to_csv('my_submission.csv',index = False, header= True)